In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
import os
import json

In [5]:
sp_conf = SparkConf()
sp_conf.setAppName('PoliticoSampler')
sp_conf.set('spark.driver.memory', '4g')
sc = SparkContext(conf=sp_conf) #'local[4]', 'PoliticoExplorer')
sqlContext = HiveContext(sc)

In [6]:
acc = os.environ['AWS_ACCESS_KEY_ID']
sec = os.environ['AWS_SECRET_ACCESS_KEY']

In [9]:
politicos = sc.textFile('data/politicos/full')
politicos.cache()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2

In [ ]:
politicos.count()

In [28]:
post_single = '''{"author":"cpcaper","first_post_id":"c2kfpnt","first_post_body":"I don't think God goes on reddit, but I guess you never know...","first_post_ups":2,"first_post_downs":0,"first_post_link_id":"t3_khkx3","parent_type":"t3","first_post_datetime":"2011-09-16 23:27:25.0","last_post_datetime":"2011-09-16 23:27:25.0","post_ids":["c2kfpnt"],"post_datetimes":["2011-09-16 23:27:25.0"],"total_posts":1}'''
post_multi = '''{"author":"picklefeather","first_post_id":"c07dg4c","first_post_body":"For five minutes, you get to be my favorite senator, McCaskill. Don't waste it.","first_post_ups":26,"first_post_downs":0,"first_post_link_id":"t3_7tpcq","first_post_responses":["The real question is will this slow moving infection of backbone spread to the rest of the Democrats? Is this a 24 hour flu or the beginnings of an epidemic?","Wasted before it even happened, as jp007 said here: http://www.reddit.com/r/politics/comments/7tpcq/congress_may_finally_have_had_enough_we_have_a/c07dhyd when s/he linked this article: http://kansascity.bizjournals.com/kansascity/stories/2008/09/29/daily23.html that says this senator voted for the bailout."],"first_post_response_ups":[12,1],"first_post_response_downs":[0,0],"first_post_total_responses":2,"first_post_avg_response_ups":6.5,"first_post_avg_response_downs":0.0,"parent_type":"t3","first_post_datetime":"2009-01-30 20:41:24.0","last_post_datetime":"2011-04-12 22:45:57.0","post_ids":["c07diek","c0hoi3g","c1p3gmg","c07dg4c","c1p3i56"],"post_datetimes":["2009-01-30 22:04:30.0","2009-12-24 12:23:59.0","2011-04-12 22:38:13.0","2009-01-30 20:41:24.0","2011-04-12 22:45:57.0"],"total_posts":5}'''

def single_post(line):
    return '"total_posts":1' in line

In [29]:
single_post(post_single)

True

In [30]:
single_post(post_multi)

False

In [31]:
# politicos_mixed_post = politicos.sample(False, 0.001)
# politicos_mixed_post.saveAsTextFile('data/politicos/politicos_mixed_sample')

In [32]:
politicos_single_post = politicos.sample(False, 0.0996).filter(single_post)
#politicos_single_post.saveAsTextFile('s3n://%s:%s@boazreddit/politicos_single_sample_large' % (acc, sec))
politicos_single_post.saveAsTextFile('data/politicos/politicos_single_sample_large')

In [ ]:
politicos_multi_post = politicos.sample(False, 0.0432).filter(lambda line: not single_post(line))
#politicos_multi_post.saveAsTextFile('s3n://%s:%s@boazreddit/politicos_multi_sample_large' % (acc, sec))
politicos_single_post.saveAsTextFile('data/politicos/politicos_single_sample_large')